## How do I make an RDD?

RDDs can be created from stable storage or by transforming other RDDs. Run the cells below to create RDDs from files on the local drive.  All data files can be downloaded from https://www.cse.ust.hk/msbd5003/data/

In [1]:
#import os
#os.environ["PYSPARK_PYTHON"]="/usr/bin/python2.7"
from pyspark import SparkContext
sc= SparkContext()
fruits = sc.textFile('fruits.txt')
yellowThings = sc.textFile('yellowthing.txt')
print fruits.collect()
print yellowThings.collect()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=PySparkShell, master=local[*]) created by <module> at /home/chris/anaconda3/envs/ipykernel_py2/lib/python2.7/site-packages/IPython/utils/py3compat.py:289 

In [2]:
from pyspark import SparkContext
sc= SparkContext()
fruits = sc.textFile('fruits.txt')
yellowThings = sc.textFile('yellowthing.txt')
print fruits.collect()
print yellowThings.collect()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-1-b7d90badc06b>:4 

In [2]:
# Read data from HDFS :
fruits = sc.textFile('fruits.txt')
fruits.collect()

NameError: name 'sc' is not defined

----------

##  RDD operations

In [6]:
# map
fruitsReversed = fruits.map(lambda fruit: fruit[::-1])

NameError: name 'fruits' is not defined

In [ ]:
# fruitsReversed = fruits.map(lambda fruit: fruit[::-1])
fruitsReversed.cache()
# try changing the file and re-execute with and without cache
fruitsReversed.collect()

In [ ]:
# filter
shortFruits = fruits.filter(lambda fruit: len(fruit) <= 5)
shortFruits.collect()

In [ ]:
# flatMap
characters = fruits.flatMap(lambda fruit: list(fruit))
characters.collect()

In [ ]:
# union
fruitsAndYellowThings = fruits.union(yellowThings)
fruitsAndYellowThings.collect()

In [ ]:
# intersection
yellowFruits = fruits.intersection(yellowThings)
yellowFruits.collect()

In [ ]:

# distinct
distinctFruitsAndYellowThings = fruitsAndYellowThings.distinct()
distinctFruitsAndYellowThings.collect()

### RDD actions
Following are examples of some of the common actions available. For a detailed list, see [RDD Actions](https://spark.apache.org/docs/2.0.0/programming-guide.html#actions).

Run some transformations below to understand this better. Place the cursor in the cell and press **SHIFT + ENTER**.

In [ ]:
# collect
fruitsArray = fruits.collect()
yellowThingsArray = yellowThings.collect()
fruitsArray

In [ ]:
# count
numFruits = fruits.count()
numFruits

In [ ]:
# take
first3Fruits = fruits.take(3)
first3Fruits

In [ ]:
# reduce
letterSet = fruits.map(lambda fruit: set(fruit)).reduce(lambda x, y: x.union(y))
letterSet

In [ ]:
letterSet = fruits.flatMap(lambda fruit: list(fruit)).distinct().collect()
letterSet

### Closure

In [ ]:
counter = 0
rdd = sc.parallelize(xrange(10))

# Wrong: Don't do this!!
def increment_counter(x):
    global counter
    counter += x

print rdd.collect()
rdd.foreach(increment_counter)

print counter

In [3]:
A = sc.parallelize(xrange(1, 1000))
t = 100
B = A.filter(lambda x: x * x > t )
t = 200
B.cache()
C = B.filter(lambda x: x * x < t )
print C.show()



AttributeError: 'PipelinedRDD' object has no attribute 'show'

In [ ]:
rdd = sc.parallelize(xrange(10))
accum = sc.accumulator(0)

def g(x):
    global accum
    accum += x

a = rdd.foreach(g)

print accum.value

In [ ]:
rdd = sc.parallelize(xrange(10))
accum = sc.accumulator(0)

def g(x):
    global accum
    accum += x
    return x * x

a = rdd.map(g)
print accum.value
print rdd.reduce(lambda x, y: x+y)
a.cache()
tmp = a.count()
print accum.value
print rdd.reduce(lambda x, y: x+y)

tmp = a.count()
print accum.value
print rdd.reduce(lambda x, y: x+y)


### Computing Pi using Monte Carlo simulation

In [ ]:
# From the official spark examples.

import sys
import random

partitions = 10
n = 1000 * partitions

def f(_):
    x = random.random()
    y = random.random()
    return 1 if x ** 2 + y ** 2 < 1 else 0

count = sc.parallelize(xrange(1, n + 1), partitions) \
          .map(f).sum()

print "Pi is roughly", 4.0 * count / n

In [ ]:
# Example: glom
import sys
import random
a = sc.parallelize(xrange(0,20),4)
print a.collect()
print a.glom().collect()
print a.map(lambda x: x+1).glom().collect()

In [ ]:
# Example: mapPartition and mapPartitionWithIndex
a = sc.parallelize(xrange(0,20),4)
print a.glom().collect()

def f(it):
    s = 0
    for i in it:
        s += i
        yield s

print a.mapPartitions(f).collect()

def f(index, it):
    s = index
    print it
    print s
    for i in it:
        s += i
        yield s

print a.mapPartitionsWithIndex(f).collect()

In [ ]:
# Correct version
import random

partitions = 10000
n = 1000 * partitions

def f(index, it):
    random.seed(index + 987236)
    for i in it:
        x = random.random()
        y = random.random()
        yield 1 if x ** 2 + y ** 2 < 1 else 0

count = sc.parallelize(xrange(1, n + 1), partitions) \
          .mapPartitionsWithIndex(f).sum()

print "Pi is roughly", 4.0 * count / n

### Closure and Persistence

In [ ]:
A = sc.parallelize(xrange(10))

x = 5
B = A.filter(lambda z: z < x)
# B.cache()
print B.take(10)
# print B.collect()
x = 3
print B.take(10) 
# print B.collect()
# collect() doesn't always re-collect data - bad design!

In [ ]:
# RDD variables are references
A = sc.parallelize(xrange(10))
B = A.map(lambda x: x*2)
A = B.map(lambda x: x+1)
A.take(10)

In [ ]:
# Linear-time selection

data = [34, 67, 21, 56, 47, 89, 12, 44, 74, 43, 26]
A = sc.parallelize(data,2)
k = 4
#  filter transformaiton is lazy not really compute, it only compute when action require result
#   first count() action 
## 
##
#
while True:
    #print A.collect()
    x = A.first()
    A1 = A.filter(lambda z: z < x)
    #print A1.collect()
    A2 = A.filter(lambda z: z > x)
    mid = A1.count()
    #print "mid = %d" % mid
    if mid == k:
        print x
        break
    
    if k < mid:
        A = A1
        #print A.collect()
    else:
        A = A2
        #print A.collect()
        k = k - mid - 1
     #print k
    #A.cache()

In [ ]:
sorted(data)

### Key-Value Pairs

In [ ]:
# reduceByKey
numFruitsByLength = fruits.map(lambda fruit: (len(fruit), 1)).reduceByKey(lambda x, y: x + y)
numFruitsByLength.collect()

In [ ]:
from operator import add

lines = sc.textFile('../data/course.txt')
counts = lines.flatMap(lambda x: x.split()) \
              .map(lambda x: (x, 1)) \
              .reduceByKey(add)
counts.sortByKey().collect()

In [ ]:
counts.sortBy(lambda x: x[1], False).collect()

In [ ]:
# Join simple example

products = sc.parallelize([(1, "Apple"), (2, "Orange"), (3, "TV"), (5, "Computer")])
#trans = sc.parallelize([(1, 134, "OK"), (3, 34, "OK"), (5, 162, "Error"), (1, 135, "OK"), (2, 53, "OK"), (1, 45, "OK")])
trans = sc.parallelize([(1, (134, "OK")), (3, (34, "OK")), (5, (162, "Error")), (1, (135, "OK")), (2, (53, "OK")), (1, (45, "OK"))])

print products.join(trans).collect()

### K-means clustering

In [ ]:
import numpy as np

def parseVector(line):
    return np.array([float(x) for x in line.split()])

def closestPoint(p, centers):
    bestIndex = 0
    closest = float("+inf")
    for i in range(len(centers)):
        tempDist = np.sum((p - centers[i]) ** 2)
        if tempDist < closest:
            closest = tempDist
            bestIndex = i
    return bestIndex

# The data file can be downloaded at http://www.cse.ust.hk/msbd5003/data/kmeans_data.txt
lines = sc.textFile('../data/kmeans_data.txt', 5)  

# The data file can be downloaded at http://www.cse.ust.hk/msbd5003/data/kmeans_bigdata.txt
# lines = sc.textFile('../data/kmeans_bigdata.txt', 5)  
# lines is an RDD of strings
K = 3
convergeDist = 0.01  
# terminate algorithm when the total distance from old center to new centers is less than this value

data = lines.map(parseVector).cache() # data is an RDD of arrays

kCenters = data.takeSample(False, K, 1)  # intial centers as a list of arrays
tempDist = 1.0  # total distance from old centers to new centers

while tempDist > convergeDist:
    closest = data.map(lambda p: (closestPoint(p, kCenters), (p, 1)))
    # for each point in data, find its closest center
    # closest is an RDD of tuples (index of closest center, (point, 1))
        
    pointStats = closest.reduceByKey(lambda p1, p2: (p1[0] + p2[0], p1[1] + p2[1]))
    # pointStats is an RDD of tuples (index of center,
    # (array of sums of coordinates, total number of points assigned))
    
    newCenters = pointStats.map(lambda st: (st[0], st[1][0] / st[1][1])).collect()
    # compute the new centers
    
    tempDist = sum(np.sum((kCenters[i] - p) ** 2) for (i, p) in newCenters)
    # compute the total disctance from old centers to new centers
    
    for (i, p) in newCenters:
        kCenters[i] = p
        
print "Final centers: ", kCenters


### PageRank

In [3]:
import re
from operator import add

def computeContribs(urls, rank):
    # Calculates URL contributions to the rank of other URLs.
    num_urls = len(urls)
    for url in urls:
        yield (url, rank / num_urls)

def parseNeighbors(urls):
    # Parses a urls pair string into urls pair."""
    parts = urls.split(' ')
    return parts[0], parts[1]

# Loads in input file. It should be in format of:
#     URL         neighbor URL
#     URL         neighbor URL
#     URL         neighbor URL
#     ...

# The data file can be downloaded at http://www.cse.ust.hk/msbd5003/data/*
lines = sc.textFile("pagerank_data.txt", 2)
# lines = sc.textFile("../data/dblp.in", 5)

numOfIterations = 10

# Loads all URLs from input file and initialize their neighbors. 
links = lines.map(lambda urls: parseNeighbors(urls)) \
             .groupByKey()

# Loads all URLs with other URL(s) link to from input file 
# and initialize ranks of them to one.
ranks = links.mapValues(lambda neighbors: 1.0)

# Calculates and updates URL ranks continuously using PageRank algorithm.
for iteration in range(numOfIterations):
    # Calculates URL contributions to the rank of other URLs.
    contribs = links.join(ranks) \
                    .flatMap(lambda url_urls_rank:
                             computeContribs(url_urls_rank[1][0],
                                             url_urls_rank[1][1]))
    # After the join, each element in the RDD is of the form
    # (url, (list of neighbor urls, rank))
    
    # Re-calculates URL ranks based on neighbor contributions.
    ranks = contribs.reduceByKey(add).mapValues(lambda rank: rank * 0.85 + 0.15)
    # ranks = contribs.reduceByKey(add).map(lambda (url, rank): (url, rank * 0.85 + 0.15))

print ranks.top(5, lambda x: x[1])

[(u'1', 1.2981882732854677), (u'3', 0.9999999999999998), (u'4', 0.9999999999999998), (u'2', 0.7018117267145316)]


### Join vs. Broadcast Variables

In [ ]:
products = sc.parallelize([(1, "Apple"), (2, "Orange"), (3, "TV"), (5, "Computer")])
trans = sc.parallelize([(1, (134, "OK")), (3, (34, "OK")), (5, (162, "Error")), (1, (135, "OK")), (2, (53, "OK")), (1, (45, "OK"))])

print trans.join(products).collect()


In [ ]:
products = {1: "Apple", 2: "Orange", 3: "TV", 5: "Computer"}
trans = sc.parallelize([(1, (134, "OK")), (3, (34, "OK")), (5, (162, "Error")), (1, (135, "OK")), (2, (53, "OK")), (1, (45, "OK"))])

broadcasted_products = sc.broadcast(products)

results = trans.map(lambda x: (x[0], broadcasted_products.value[x[0]], x[1]))
#  results = trans.map(lambda x: (x[0], products[x[0]], x[1]))
print results.collect()
